In [ ]:
import numpy
import requests
from bs4 import BeautifulSoup


git_hub_url = 'https://gist.github.com/paulmillr/2657075'
base_Git_hub_url = "https://github.com/"
starred_Git_hub_user_url = 'https://api.github.com/users/<user>/repos'


Params = {
   
}

Headers = {
   
}

# Récupère les noms des contributeurs GitHub les mieux classés du site Gist GitHub;

def get_names(token):
    url = git_hub_url
    data = requests.get(url)

    results = []
    parser = BeautifulSoup(data.text, 'html.parser')
    article = parser.find("article")
    if article:
        table = article.find("table")
        if table:
            for a_node in table.findAll("a"):
                url = a_node.attrs['href']
                if url.startswith(base_Git_hub_url):
                    results.append(url[len(base_Git_hub_url):])
    return results


# Récupère la moyenne 'stargazers_count' des référentiels avec lesquels l'utilisateur désigné est enregistré.
def get_starred_repos(user, token):
    url = starred_Git_hub_user_url.replace('<user>', user)
    # Params['access_token'] = token
    Headers['Authorization'] = 'token ' + token
    r = requests.get(url, params=Params, headers=Headers)
    data = r.json()

    results = [0];
    for i in data:
        if isinstance(i, dict):
            results.append(i["stargazers_count"])
    return numpy.mean(results)

# Récupère et classe les contributeurs GitHub les mieux notés en fonction de leur moyenne 'stargazers_count';

def sort_users_by_starred_repos(token):
    results = {}
    for user in get_names(token):
        stars = get_starred_repos(user, token)
       
        results[user] = stars
    return sorted(results.items(), key=lambda x: x[-1], reverse=True)

# Charge un jeton oAuth à partir du fichier désigné


def load_oauth_token(filename):
    with open(filename, 'r') as file:
        token = file.readline().strip()
        return token

# Imprime les résultats fournis
def pretty_print(results):
    for username, average in results:
        print("User: %s = Average StarGazers: %2f" % (username, average))

# Test
pretty_print(sort_users_by_starred_repos(load_oauth_token('GitHub_API_OAuth_token.txt')))